<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/PP/make_triangle_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna > /dev/null
!pip install pyproj > /dev/null
!pip install simdkalman > /dev/null
    
import os
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns

import optuna
import plotly
import plotly.express as px
import pyproj
from pathlib import Path
from pyproj import Proj, transform
from tqdm.notebook import tqdm
import simdkalman

In [2]:
path = Path("/content/drive/MyDrive/GSDC")
#test_base = pd.read_csv(path / "baseline_locations_test.csv")
test_base = pd.read_csv("/content/drive/MyDrive/GSDC/test_pre_next_move_closer_SJC.csv")

sub = pd.read_csv(path / "sample_submission.csv")

truths = (path / "train").rglob("ground_truth.csv")

df_list = []
cols = ["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]

for t in tqdm(truths, total=73):
    df_phone = pd.read_csv(t, usecols=cols)
    df_list.append(df_phone)
df_truth = pd.concat(df_list, ignore_index=True)

train_base = pd.read_csv(path / "baseline_locations_train.csv")
all_df = df_truth.merge(train_base, how="inner", on=cols[:3], suffixes=("_truth", '_train_base'))

In [3]:
def get_groundtruth(path: Path) -> pd.DataFrame:
        output_df = pd.DataFrame()
        
        for path in glob(str(path / 'train/*/*/ground_truth.csv')):
            _df = pd.read_csv(path)
            output_df = pd.concat([output_df, _df])
        output_df = output_df.reset_index(drop=True)
        
        _columns = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']
        output_df[['t_'+col for col in _columns]] = output_df[_columns]
        output_df = output_df.drop(columns=_columns, axis=1)
        return output_df

train_base = train_base.merge(
    get_groundtruth(path),
    on=['collectionName', 'phoneName', 'millisSinceGpsEpoch']
)

In [54]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    return output_df

In [4]:
def calc_haversine(lat1, lon1, lat2, lon2):
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    d = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(d**0.5)
    return dist

In [200]:
def make_triangle_train(input_df):
    output_df = input_df.copy()

    def add_triangle_features(input_df):
        output_df = input_df.copy()
        output_df["latDeg_pro_1"] = output_df["latDeg"].shift(-1)
        output_df["latDeg_pro_2"] = output_df["latDeg"].shift(-2)
        output_df["lngDeg_pro_1"] = output_df["lngDeg"].shift(-1)
        output_df["lngDeg_pro_2"] = output_df["lngDeg"].shift(-2)
        output_df["millisSinceGpsEpoch_pro_1"] = output_df["millisSinceGpsEpoch"].shift(-1)
        output_df["millisSinceGpsEpoch_pro_2"] = output_df["millisSinceGpsEpoch"].shift(-2)
        output_df["latDeg_mean_point"] = (output_df["latDeg"] + ((output_df["latDeg_pro_2"] - output_df["latDeg"]) * 
                                                               ((output_df["millisSinceGpsEpoch_pro_1"] - output_df["millisSinceGpsEpoch"]) /
                                                               (output_df["millisSinceGpsEpoch_pro_2"] - output_df["millisSinceGpsEpoch"])))).shift(1)
        
        output_df["lngDeg_mean_point"] = (output_df["lngDeg"] + ((output_df["lngDeg_pro_2"] - output_df["lngDeg"]) * 
                                                               ((output_df["millisSinceGpsEpoch_pro_1"] - output_df["millisSinceGpsEpoch"]) /
                                                               (output_df["millisSinceGpsEpoch_pro_2"] - output_df["millisSinceGpsEpoch"])))).shift(1)


        degree_list = []
        for lat, lng, lat_1, lng_1, lat_2, lng_2 in zip(
            output_df["latDeg"].to_numpy(),
            output_df["lngDeg"].to_numpy(),
            output_df["latDeg_pro_1"].to_numpy(),
            output_df["lngDeg_pro_1"].to_numpy(),
            output_df["latDeg_pro_2"].to_numpy(),
            output_df["lngDeg_pro_2"].to_numpy()
        ):
            p0 = np.array([lat, lng])
            p1 = np.array([lat_1, lng_1])
            p2 = np.array([lat_2, lng_2])
            
            vec_p0 = p0 - p1
            vec_p2 = p2 - p1
            length_vec_p0 = np.linalg.norm(vec_p0)
            length_vec_p2 = np.linalg.norm(vec_p2)
            inner = np.inner(vec_p0, vec_p2)
            degree = np.rad2deg(np.arccos(inner / (length_vec_p0 * length_vec_p2)))
            degree_list.append(degree)
        degree_list.insert(0, 180)
        degree_list.pop(-1)
        degree_list[-1] = 180
        output_df["degree"] = degree_list
        return output_df

    train_SJC_list = ["2021-04-22-US-SJC-1", "2021-04-28-US-SJC-1", "2021-04-29-US-SJC-2"]
    lat_list = []
    lng_list = []

    for collection in output_df["collectionName"].unique():
        collection_df = output_df[output_df["collectionName"] == collection]
        if collection in train_SJC_list:
            for phone in collection_df["phoneName"].unique():
                phone_df = collection_df[collection_df["phoneName"] == phone]
                triangle_df = add_triangle_features(phone_df)
                for lat, lng, lat_mp, lng_mp, deg in zip(
                    triangle_df["latDeg"].to_numpy(),
                    triangle_df["lngDeg"].to_numpy(),
                    triangle_df["latDeg_mean_point"].to_numpy(),
                    triangle_df["lngDeg_mean_point"].to_numpy(),
                    triangle_df["degree"].to_numpy()
                ):
                    if deg < 140:
                        lat_f = (lat + lat_mp)/2
                        lng_f = (lng + lng_mp)/2
                        lat_list.append(lat_f)
                        lng_list.append(lng_f)
                    else:
                        lat_list.append(lat)
                        lng_list.append(lng)
        else:
            for lat, lng in zip(
                collection_df["latDeg"].to_numpy(),
                collection_df["lngDeg"].to_numpy(),
            ):
                lat_list.append(lat)
                lng_list.append(lng)
    output_df["latDeg"] = lat_list
    output_df["lngDeg"] = lng_list
    return output_df

In [201]:
a = make_triangle_train(train_base)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: RuntimeWarning:

invalid value encountered in double_scalars



In [190]:
base = check_score(train_base)

error meter: 3.846848374990627
score: 5.287970649084159


In [193]:
_90 = check_score(a)

error meter: 3.8027141766770223
score: 5.242640175801573


In [196]:
_110 = check_score(a)

error meter: 3.7981849849982394
score: 5.238530893540435


In [199]:
_120 = check_score(a)

error meter: 3.7956901494229087
score: 5.2352358059150434


In [202]:
_140 = check_score(a)

error meter: 3.7912517142085713
score: 5.234516597130473


In [25]:
a["degree"].describe()

count    131124.000000
mean        148.272650
std          51.138589
min           0.000000
25%         147.943917
50%         174.732489
75%         178.159469
max         180.000000
Name: degree, dtype: float64

In [33]:
fig = px.scatter_mapbox(train_base[train_base["phone"] == "2021-04-29-US-SJC-2_SamsungS20Ultra"],
                            
                        # Here, plotly gets, (x,y) coordinates
                        lat="latDeg",
                        lon="lngDeg",
                            
                        #Here, plotly detects color of series
                        color="phoneName",
                        labels="phoneName",
                            
                        zoom=400,
                        height=600,
                        width=800)
fig.update_layout(mapbox_style='stamen-terrain')
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_layout(title_text="GPS trafic")
fig.show()

In [ ]:
a = np.array([0,1,2])
    b = np.array([10,20,30])
    c = np.array([5,7,9])

    vec_a = a - b
    vec_c = c - b
    length_vec_a = np.linalg.norm(vec_a)
    length_vec_c = np.linalg.norm(vec_c)
    inner_product = np.inner(vec_a, vec_c)
    degree = np.rad2deg(np.arccos(inner_product / (length_vec_a * length_vec_c)))

In [ ]:
def mean_prediction_train(input_df):
    input_df["phone"] = input_df["collectionName"] + "_" + input_df["phoneName"]
    
    def add_distance_diff(df):
        df['latDeg_pre'] = df['latDeg'].shift(1)
        df['latDeg_pro'] = df['latDeg'].shift(-1)
        df['lngDeg_pre'] = df['lngDeg'].shift(1)
        df['lngDeg_pro'] = df['lngDeg'].shift(-1)
        df['phone_pre'] = df['phone'].shift(1)
        df['phone_pro'] = df['phone'].shift(-1)
        
        df['dist_pre'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_pre'], df['lngDeg_pre'])
        df['dist_pro'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_pro'], df['lngDeg_pro'])
        
        df.loc[df['phone']!=df['phone_pre'], ['latDeg_pre', 'lngDeg_pre', 'dist_pre']] = np.nan
        df.loc[df['phone']!=df['phone_pro'], ['latDeg_pro', 'lngDeg_pro', 'dist_pro']] = np.nan
        
        return df


    def make_lerp_data(input_df):
        org_colus = input_df.columns

        time_list = input_df[["collectionName", "millisSinceGpsEpoch"]].drop_duplicates()
        phone_list = input_df[["collectionName", "phoneName"]].drop_duplicates()
        tmp = time_list.merge(phone_list, on="collectionName", how="outer")

        output_df = tmp.merge(input_df, on=["collectionName", "millisSinceGpsEpoch", "phoneName"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        output_df = output_df.sort_values(["phone", "millisSinceGpsEpoch"])

        output_df["latDeg_pre"] = output_df["latDeg"].shift(1)
        output_df["latDeg_pro"] = output_df["latDeg"].shift(-1)
        output_df["lngDeg_pre"] = output_df["lngDeg"].shift(1)
        output_df["lngDeg_pro"] = output_df["lngDeg"].shift(-1)
        output_df["phone_pre"] = output_df["phone"].shift(1)
        output_df["phone_pro"] = output_df["phone"].shift(-1)
        output_df["millisSinceGpsEpoch_pre"] = output_df["millisSinceGpsEpoch"].shift(1)
        output_df["millisSinceGpsEpoch_pro"] = output_df["millisSinceGpsEpoch"].shift(-1)

        output_df = output_df[(output_df["latDeg"].isnull())&(output_df["phone"] == output_df["phone_pre"])&
                            (output_df["phone"] == output_df["phone_pro"])].copy()

        #preとproの間を経過時間を考慮して算出
        output_df["latDeg"] = output_df["latDeg_pre"] + ((output_df["latDeg_pro"] - output_df["latDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        output_df["lngDeg"] = output_df["lngDeg_pre"] + ((output_df["lngDeg_pro"] - output_df["lngDeg_pre"]) * 
                                                        ((output_df["millisSinceGpsEpoch"] - output_df["millisSinceGpsEpoch_pre"]) /
                                                        (output_df["millisSinceGpsEpoch_pro"] - output_df["millisSinceGpsEpoch_pre"])))
        
        output_df = output_df[~output_df['latDeg'].isnull()]

        return output_df[org_colus]

    
    def calc_mean_pred(input_df, lerp_df):
        input_df["phone"] = input_df["collectionName"] + "_" + input_df["phoneName"]
        add_lerp = pd.concat([input_df, lerp_df])
        mean_pred_result = add_lerp.groupby(["collectionName", "millisSinceGpsEpoch"])[["latDeg", "lngDeg"]].mean().reset_index()
        output_df = input_df[["collectionName", "phoneName", "millisSinceGpsEpoch"]].copy()
        output_df = output_df.merge(mean_pred_result[["collectionName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]],
                                        on=["collectionName", "millisSinceGpsEpoch"], how="left")
        output_df["phone"] = output_df["collectionName"] + "_" + output_df["phoneName"]
        return output_df

    input_df_ = add_distance_diff(input_df)
    th = 50
    input_df_.loc[((input_df_['dist_pre'] > th) & (input_df_['dist_pro'] > th)), ['latDeg', 'lngDeg']] = np.nan

    #input_df_kalman = kalman_filter_train(input_df_)
    
    lerp = make_lerp_data(input_df_)
    mean_pred  = calc_mean_pred(input_df_, lerp)

    output_df = kalman_filter_train(mean_pred)
    output_df["t_latDeg"] = input_df["t_latDeg"]
    output_df["t_lngDeg"] = input_df["t_lngDeg"]
    
    return output_df